## Pre-processing

In [240]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import glob
from numpy import array
from numpy import argmax
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
pd.options.mode.chained_assignment = None
data = pd.read_csv("data_hack.csv")

Dropping unnecessary columns

In [241]:
train_data = data.drop(['Sr','club','nationality','pac','sho','pas','dri','def','phy',\
    'international_reputation','skill_moves','weak_foot','work_rate_att','work_rate_def',\
    'preferred_foot','crossing','finishing','heading_accuracy','volleys','dribbling','curve',\
    'free_kick_accuracy','acceleration','sprint_speed','agility','reactions','balance','shot_power',\
    'jumping','stamina','strength','long_shots','aggression','penalties','composure','standing_tackle',\
    'sliding_tackle','gk_diving','gk_kicking','gk_positioning','CS_part-12/13','CS_part-13/14',\
    'CS_part-14/15','CS_part-15/16','CS_part-16/17','Sub_p-16/17','Sub_p-15/16','Sub_p-14/15','Sub_p-13/14',\
    'Sub_p-12/13','CO-12/13','CO-13/14','CO-14/15','CO-15/16','CO-16/17','Conceded_1+-12/13','Conceded_1+-13/14',\
    'Conceded_1+-14/15','Conceded_1+-15/16','Conceded_1+-16/17','Apps-12/13','Apps-13/14','Apps-14/15','Apps-15/16',\
    'Apps-16/17','Form-12/13','Form-13/14','Form-14/15','Form-15/16','Form-16/17'],axis=1)

In [242]:
train_data = train_data.drop(train_data.columns[40:],axis=1)
train_data.to_csv('check.csv',index=None)

In [243]:
rows = train_data.league.size
print (rows)

3922


Merging of players with duplicate rows. For example: Player A played as midfielder in Season 1 and forward in the latest.He then had two rows. We merged them into one row with his position same as that of the last season in which he played.

In [244]:
count=0
l=[]
for i in range(rows):
    if(i+1<rows):
        if(train_data.iloc[i].Players == train_data.iloc[i+1].Players):
            temp_df = train_data[train_data.columns[13:]].iloc[i] + train_data[train_data.columns[13:]].iloc[i+1]
            for j in range(25):
                train_data[train_data.columns[13+j]].iloc[i] = temp_df[j]
            l.append(i+1)

In [245]:
train_data = train_data.drop(train_data.index[l])
train_data = train_data.drop(['Players'],axis=1)

In [246]:
rows = train_data.league.size
print (rows)

3891


If there are no statistics of a player for the two seasons. We remove the player.

In [247]:
l = []
for i in range(rows):
    flag = 0
    for j in range(26):
        if(train_data[train_data.columns[12+j]].iloc[i]!=0):
            flag=1
    if(flag==0):
        l.append(i)

In [248]:
train_data = train_data.drop(train_data.index[l])
print(train_data.league.size)

3891


There were many leagues like Turkish League, Belgian League etc, for our purpose we have used only the top few 
leagues such as English Premier League, Spanish Premier League etc.

In [249]:
leagues = train_data.league.unique()

l = []
for i in leagues:
    size = train_data.loc[train_data['league'] == i].shape[0]
    if(size > 70):
        l.append(i)
        
rows = train_data.league.size
rm=[]

for i in range(rows):
    if train_data['league'].iloc[i] not in l:
        rm.append(i)

train_data = train_data.drop(train_data.index[rm])

print(train_data.league.size)

3147


In [250]:
y = train_data['overall']
y.to_csv("ground_truth/overall.csv",index=None)
goalie_y = y.loc[train_data['Position'] == 'Goalkeeper']
defend_y = y.loc[train_data['Position'] == 'Defender']
mid_y = y.loc[train_data['Position'] == 'Midfielder']
att_y = y.loc[train_data['Position'] == 'Forward']

In [251]:
print(goalie_y.size, defend_y.size, mid_y.size, att_y.size)

339 958 1196 654


In [252]:
print (y.describe())

count    3147.000000
mean       72.318081
std         6.407605
min        48.000000
25%        68.000000
50%        72.000000
75%        77.000000
max        94.000000
Name: overall, dtype: float64


In [253]:
train_data = train_data.drop(['overall','Points-16/17','Points-15/16'],axis=1)
train_data.describe()

,age,short_passing,long_passing,ball_control,interceptions,positioning,vision,marking,gk_handling,gk_reflexes,...,Goals-15/16,A-15/16,CS-15/16,Yellow-15/16,Red-15/16,Off-15/16,Pen_SV-15/16,Pen_M-15/16,Goals_conceded-15/16,OG-15/16
count,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,...,3147.000000,3147.000000,3147.000000,3147.000000,3147.000000,3147.00000,3147.000000,3147.000000,3147.000000,3147.000000
mean,25.769940,64.472831,58.324754,64.666349,51.361932,54.836034,58.042898,47.598983,16.957102,17.460121,...,1.158881,0.816015,1.111217,1.742930,0.045440,2.36225,0.019384,0.025739,5.794725,0.037178
std,4.490699,15.185465,15.602766,17.567088,22.640904,21.180298,15.589142,23.929516,18.711848,20.209297,...,3.089498,1.863293,2.936447,2.716589,0.218722,4.06726,0.181659,0.187766,13.245822,0.205340
min,16.000000,11.000000,10.000000,10.000000,5.000000,4.000000,10.000000,5.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,22.000000,60.000000,50.000000,62.000000,28.000000,44.000000,48.000000,23.000000,8.000000,8.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,68.000000,62.000000,70.000000,58.000000,61.000000,60.000000,54.000000,11.000000,11.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,29.000000,74.000000,69.000000,75.000000,71.000000,70.000000,70.000000,70.000000,14.000000,14.000000,...,1.000000,1.000000,0.000000,3.000000,0.000000,3.00000,0.000000,0.000000,0.000000,0.000000
max,40.000000,92.000000,93.000000,95.000000,92.000000,95.000000,92.000000,92.000000,91.000000,150.000000,...,40.000000,19.000000,24.000000,17.000000,2.000000,30.00000,4.000000,3.000000,72.000000,2.000000


In [254]:
train_data = pd.get_dummies(train_data, columns=["league"])
# train_data.describe()

In [255]:
    goalie = train_data.loc[train_data['Position'] == 'Goalkeeper']
    defend = train_data.loc[train_data['Position'] == 'Defender']
    mid = train_data.loc[train_data['Position'] == 'Midfielder']
    att = train_data.loc[train_data['Position'] == 'Forward']
    goalie = goalie.drop(['Position'],axis=1)
    defend = defend.drop(['Position'],axis=1)
    mid = mid.drop(['Position'],axis=1)
    att = att.drop(['Position'],axis=1)
    print(goalie.age.size, defend.age.size, mid.age.size, att.age.size)

339 958 1196 654


In [256]:
train_data.to_csv('training_new/training_data.csv',index=None)
goalie.to_csv('training_new/goalie.csv',index=None)
defend.to_csv('training_new/defender.csv',index=None)
mid.to_csv('training_new/midfielder.csv',index=None)
att.to_csv('training_new/forward.csv',index=None)
# goalie_y.to_csv('ground_truth/goalie_y.csv',index=None)
# defend_y.to_csv('ground_truth/defender_y.csv',index=None)
# mid_y.to_csv('ground_truth/midfielder_y.csv',index=None)
# att_y.to_csv('ground_truth/forward_y.csv',index=None)